In [1]:
using PyPlot, Printf, Random, DelimitedFiles, Dates

include("decode.jl")
include("utils.jl")
include("updateRule.jl");

In [2]:
Random.seed!(1234);

In [3]:
function computeSBFAndTheta(rule, x_t, col)
   return rule.computeSBF(x_t, col, alpha_0[:,col])
end

function decodeSurprise(seq, m, alpha_0, rule, ignoreFirstM = true)
    function computeSBFAndTheta(rule, x_t, col)
        sbf = rule.computeSBF(x_t, col, alpha_0[:,col])
        theta = 1 - rule.computeTheta(col)
        return (sbf, theta) 
    end
    
    # define callback to compute SBF and Thetas
    returnType = Tuple{Float64, Float64}
    callback = Callback(computeSBFAndTheta, returnType)

    # decode sequence
    values = decode(seq, m, alpha_0, rule;
                    callback = callback, ignoreFirstM = ignoreFirstM)
    
    if ignoreFirstM && m > 0
        values[1:m] .= Ref{returnType}((1.0, 0.5))
    end
    
    return values
end;

In [4]:
function generateSequenceBase(chunks, ps)
    seq = Array{Int32}(undef, sum(chunks))
    prob = Array{Float32}(undef, sum(chunks))
    
    i = 1
    for (p, chunk) in zip(ps, chunks)
        for j = 1:chunk
            seq[i] = rand() < p
            prob[i] = p
            i += 1
        end
    end
    
    return (seq, prob)
end;

In [5]:
function generateSequenceTrans(chunks, ps1g2, ps2g1)
    seq = Array{Int32}(undef, sum(chunks))
    prob1g2 = Array{Float32}(undef, sum(chunks))
    prob2g1 = Array{Float32}(undef, sum(chunks))
    
    i = 2
    
    # set inital value
    seq[1] = rand() > 0.5
    prob1g2[1] = ps1g2[1]
    prob2g1[1] = ps2g1[1]
    
    for (p1g2, p2g1, chunk) in zip(ps1g2, ps2g1, chunks)
        for j = 1:chunk
            p = seq[i - 1] == 0 ? p2g1 : 1 - p1g2
            seq[i] = rand() < p
            prob1g2[i] = p1g2
            prob2g1[i] = p2g1
            i += 1
            if i > sum(chunks)
               break 
            end
        end
    end
    
    return (seq, prob1g2, prob2g1)
end;

In [6]:
function exportSeq(seq, filename)
    f = open(filename, "w")

    for el in seq
        print(f, el)
        print(f, " ")
    end
    
    close(f)
end
;

In [7]:
function readThetasMatlab(filename, len)
   f = open(filename, "r")
    t = readdlm(f)
    close(f)
    return t[1,1:len]
end;

In [8]:
function readSeq(filename, len)
    f = open(filename, "r")
    t = readdlm(f)
    close(f)
    
    return t[1,1:len]
end;

In [49]:
# ============== DECODE BASE RATE ======================
# generate sequence
chunks = [200, 200, 200, 200, 200]
len = sum(chunks)

p = 0.25
ps = [p, 1-p, p, 1-p, p]

seq_base = readSeq("../test/seq1.txt", 1000)
# (seq_base, prob) = generateSequenceBase(chunks, ps)
# exportSeq(seq_base, "../test/seq1.txt")
# exportSeq(prob, "../test/seq1_p.txt")

"""
# matlab code

s = dlmread('seq1.txt') + 1; % add one because they work with elements 1 and 2
gen_p1 = dlmread('seq1_p.txt');

% compute ...

fid = fopen('seq1_leaky_10_theta.txt','wt'); % change filename
fprintf(fid, '%.9f ', out.FIX.p1_mean');
fclose(fid);

return;
"""
;

In [50]:
values = decodeSurprise(seq_base, 0, ones(2, 1), perfect());

# here we do 2:end
thetas = map(t -> t[2], values[2:end]);
thetas_matlab = readThetasMatlab("../test/seq1_perfect_theta.txt", len)

threshold = 1e-8
n = sum(isless.(abs.(thetas - thetas_matlab), threshold))

# percentage of correct predictions
@show(n / len);

n / len = 1.0


In [36]:
thetas_matlab

1000-element Array{Any,1}:
 0.666666667
 0.75       
 0.8        
 0.833333333
 0.714285714
 0.75       
 0.777777778
 0.8        
 0.818181818
 0.833333333
 0.846153846
 0.785714286
 0.8        
 ⋮          
 0.561049445
 0.560483871
 0.560926485
 0.561368209
 0.561809045
 0.562248996
 0.562688064
 0.562124248
 0.562562563
 0.563      
 0.563436563
 0.562874251

In [52]:
thetas

1000-element Array{Float64,1}:
 0.6666666666666667
 0.75              
 0.8               
 0.8333333333333334
 0.7142857142857143
 0.75              
 0.7777777777777778
 0.8               
 0.8181818181818181
 0.8333333333333334
 0.8461538461538461
 0.7857142857142857
 0.8               
 ⋮                 
 0.5610494450050454
 0.560483870967742 
 0.5609264853977844
 0.5613682092555332
 0.5618090452261306
 0.5622489959839357
 0.5626880641925778
 0.5621242484969939
 0.5625625625625625
 0.563             
 0.5634365634365635
 0.562874251497006 

In [34]:
seq_base

1000-element Array{Any,1}:
 0
 0
 0
 0
 1
 0
 0
 0
 0
 0
 0
 1
 0
 ⋮
 0
 1
 0
 0
 0
 0
 0
 1
 0
 0
 0
 1

In [31]:
values = decodeSurprise(seq_base, 0, ones(2, 1), leaky(10));

thetas = map(t -> t[2], values[2:end]);
thetas_matlab = readThetasMatlab("../test/seq1_leaky_10_theta.txt", len)

threshold = 1e-8
n = sum(isless.(abs.(thetas - thetas_matlab), threshold))

# percentage of correct predictions
@show(n / len);

n / len = 1.0


In [32]:
thetas_matlab

1000-element Array{Any,1}:
 0.655746654
 0.731440394
 0.776005051
 0.805246899
 0.668218298
 0.710855857
 0.743493135
 0.769164098
 0.789790802
 0.806650193
 0.820623935
 0.727668166
 0.750443185
 ⋮          
 0.755442211
 0.691821395
 0.712879451
 0.731933568
 0.749174446
 0.764774637
 0.778890274
 0.71303808 
 0.732077101
 0.74930432 
 0.764892152
 0.700372056

In [45]:
# ============== DECODE TRANSITION RATE ======================
# generate sequence
chunks = [100, 200, 100, 200]
len = sum(chunks)

p = 0.25
p1g2  = 1 .- [p, 1-p, p, 1-p]
p2g1  =      [p, 1-p, p, 1-p]

seq_trans = readSeq("../test/seq2.txt", 600);
# (seq_trans, prob1g2, prob2g1) = generateSequenceTrans(chunks, p1g2, p2g1)
# exportSeq(seq, "seq2.txt")
# exportSeq(prob1g2, "seq2_p1g2.txt")
# exportSeq(prob2g1, "seq2_p2g1.txt")

In [46]:
rule = perfect()
values = decodeSurprise(seq_trans, 1, ones(2, 2), rule)

# here we do 2:end because we want to skip the first value (incomplete window)
thetas = map(t -> t[2], values[2:end]);
thetas_matlab = readThetasMatlab("../test/seq2_perfect_theta.txt", len)

threshold = 1e-8
n = sum(isless.(abs.(thetas - thetas_matlab), threshold))

# percentage of correct predictions
@show(n / len);

n / len = 1.0


In [17]:
rule = leaky(16, true)
values = decodeSurprise(seq_trans, 1, ones(2, 2), rule);

thetas = map(t -> t[2], values[2:end]);
thetas_matlab = readThetasMatlab("../test/seq2_leaky_16_theta.txt", len)

threshold = 1e-8
n = sum(isless.(abs.(thetas - thetas_matlab), threshold))

# percentage of correct predictions
@show(n / len);

n / len = 1.0


In [ ]:
"""
In the case m = 0 (item frequency learning), Maheu et al. estimate the thetas after seeing the value.

In the case m = 1 (transition frequency learning), Maheu et al. estimate the thetas before seeing the value, 
but discard the first value since there is no window before it.
"""
;

In [1]:
using BenchmarkTools

In [3]:
fn = (x) -> x.^2

function iterate2(mat)
    res = similar(mat)
    for i = 1:size(mat,2)
        for j = 1:size(mat,3)
            res[:,i,j] = fn(mat[:,i,j])
        end
    end
end

function iterate2d2(mat)
    res = similar(mat)
    for i = 1:size(mat,1)
        for j = 1:size(mat,2)
            res[i,j] = fn(mat[i,j])
        end
    end
end

function iterate3(mat)
    res = similar(mat)
    for i = 1:size(mat,1)
        for j = 1:size(mat,2)
            res[i,j,:] = fn(mat[i,j,:])
        end
    end
end

function iterate2d3(mat)
    res = similar(mat)
    for i = 1:size(mat,2)
        for j = 1:size(mat,1)
            res[j,i] = fn(mat[j,i])
        end
    end
end

function iterate4(mat)
    res = similar(mat)
    for i = eachindex(res)
        res[i] = fn(mat[i])
    end
end

N = 100
mat = rand(N,N,N)
mat2d = rand(N,N)

@btime iterate2(mat)
@btime iterate3(mat)

  6.737 ms (30002 allocations: 24.87 MiB)
  12.553 ms (30002 allocations: 24.87 MiB)
